In [1]:
from models.MTL_IBA import MTL_IBA, MTL_IBA_h3, MTL_IBA_cross, MTL_IBA_cross2, MTL_IBA_cross3
from models.MLT_net import MTL_classic
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import load_model, Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
import cv2



import os


# weight_path = 'MTL_IBA_cross3_256.h5'

# base_model = MTL_IBA_cross3(256, 256, 3, nClasses=2)
# # model = MTL_IBA_cross(img_size, img_size, depth, nClasses=2)
# # model = MTL_IBA_cross2(img_size, img_size, depth, nClasses=2)
# # model = MTL_IBA_cross3(img_size, img_size, depth, nClasses=2)

# base_model.load_weights(weight_path, by_name=True)
# base_model.summary()



Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
def load_img_preprocess(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img) 
    img = np.expand_dims(img, axis=0) # 
    img = preprocess_input(img) #
    return img


def gradient_compute(model, img):
  
    preds = model.predict(img)
    num = np.argmax(preds[0]) #
    
    model2 = Model(inputs=model.input, outputs=model.get_layer('block5_c_conv3').output)

    output = model2.output[0][:, num]
    last_layer = model2.get_layer('block5_c_conv3')
    grads = K.gradients(output, last_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2)) #
    iterate = K.function([model2.input], [pooled_grads, last_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img])

    for i in range(pooled_grads.shape[0]):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    return conv_layer_output_value

def plot_heatmap(conv_layer_output_value, img_in_path, img_out_path):
    

    mean = np.mean(conv_layer_output_value, axis=0)
    maxi = np.max(conv_layer_output_value)
    mini = np.min(conv_layer_output_value)
#     conv_layer_output_value = (conv_layer_output_value - mean)/(maxi-mini)
#     conv_layer_output_value = (conv_layer_output_value - mean)
    heatmap = np.mean(conv_layer_output_value,axis = -1)
    heatmap = np.maximum(heatmap, 0)
    mean = np.mean(heatmap, axis =0)
    heatmap = heatmap / np.max(heatmap)

    img = cv2.imread(img_in_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimopsed_img = heatmap * 0.5 + img
    
    cv2.imwrite(img_out_path, superimopsed_img)
    
    print(img_out_path)

    
def plot_dict_img_heatmap(testX_dir, save_dir, model):
    
    class_list = os.listdir(testX_dir)
    for class_name in class_list:
        class_path = os.path.join(testX_dir, class_name)
        img_list = os.listdir(class_path)
        for image_Name in img_list:
            
            img_path = os.path.join(class_path, image_Name)
            print(img_path)
            img_out_path = os.path.join(save_dir, image_Name)
            
            img = load_img_preprocess(img_path, (img_size, img_size))
            conv_value = gradient_compute(model, img)
            plot_heatmap(conv_value, img_path, img_out_path)
            

 

In [7]:
import os
from models.MTL_IBA import MTL_IBA, MTL_IBA_h3, MTL_IBA_cross, MTL_IBA_cross2, MTL_IBA_cross3
from models.MLT_net import MTL_classic
from models.MTL_Attention import MTL_Attention_model

Name =  'MTL_GCT2'
img_size =224
depth = 3
testX_dir = 'dataset/Dataset_BUSI_AN/test/images/'
save_dir = 'dataset/prediction/Dataset_BUSI_AN/' + Name
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
    

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
   
# model = MTL_IBA_cross3(img_size, img_size, depth, nClasses=2)
model = MTL_Attention_model(img_size, img_size, depth, nClasses=2)
model.load_weights('MTL_GCT2.h5', by_name=True)



plot_dict_img_heatmap(testX_dir, save_dir, model)  

dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (51).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (51).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (299).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (299).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (167).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (167).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (220).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (220).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (297).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (297).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (425).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (425).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (57).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/benign (57).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (38).png
dataset/prediction/Dataset_BUSI_AN/MTL_GCT

dataset/prediction/Dataset_BUSI_AN/MTL_GCT2/malignant (136).png
